<a href="https://colab.research.google.com/github/sihyeon3523/Six_of_cells/blob/modeling/Seunguk_(3)_lyricModelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>
<br>

# 0. Install & import libraries 

In [1]:
from IPython.display import clear_output 

!pip install transformers==4.8.2
!pip install sentencepiece==0.1.96
!pip install tensorflow_addons

clear_output()

import sklearn
import tensorflow
import transformers
import tensorflow_addonss

print(sklearn.__version__) # 1.0.2
print(tensorflow.__version__) # 2.8.0
print(transformers.__version__) # 4.8.2
print(tensorflow_addons.__version__) # 0.16.1

import sentencepiece

import pandas as pd
import numpy as np

import os
import re
import pickle 
import dill # for saving a function as a file(.pkl)
import logging # for changing the tf's logging level
import urllib.request
from tqdm import tqdm

from sklearn import model_selection
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow_addons as tfa # for using Rectified-Adam optimizer (instead of Adam optimizer) 
from tensorflow.keras import layers, initializers, losses, optimizers, metrics, callbacks 

import transformers
from transformers import TFBertModel # BertTokenizer 제외

import sentencepiece as spm
import ast  # str list to list

ModuleNotFoundError: ignored

In [ ]:
# Random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

# Transformers logging level 변경 (WARNING -> ERROR) @ https://huggingface.co/transformers/main_classes/logging.html
transformers.logging.set_verbosity(transformers.logging.ERROR)

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
#KoBERT 불러오기
!git clone https://github.com/monologg/KoBERT-Transformers.git
!mv KoBERT-Transformers/kobert_transformers/tokenization_kobert.py /content
clear_output() # clear the output

#토크나이저 생성
from tokenization_kobert import KoBertTokenizer 

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

# Import Google Drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.getcwd()

In [ ]:
# 경로만 바꾸면 됨!
path = '/content/drive/MyDrive/likelion/proj_fin_seunguk'# google drive project path
data_path = 'colab_data/temp_data/' # model data path
checkpoint_path = 'colab_data/temp_data/saved_models/' # saved model path

os.chdir(path)

# 1. Load the Data

In [ ]:
df_allsongs_original = pd.read_csv('df_allsongs.csv', lineterminator='\n')

In [ ]:
df_allsongs_original.head(3)

In [ ]:
df_allsongs = df_allsongs_original[["Title", "Artist", "Date", "Genre", "preprocess_Lyric_ver3"]]
df_allsongs

In [ ]:
print(df_allsongs.info())
print(df_allsongs.describe())

In [ ]:
df_allsongs.preprocess_Lyric_ver3.iloc[12]

Load saved Model


In [ ]:
# 1) Load the Model-builder (function)
with open(data_path + 'model_BERTfunction_v1.pkl', 'rb') as f:
    create_model = dill.loads(pickle.load(f)) # use dill to pickle a python function

# 2) Load the Bert-tokenizer 
with open(data_path + 'tokenizer-bert.pkl', 'rb') as f:
    tokenizer = pickle.load(f) 


# 3) Create the model & load the Model-weights (from checkpoint file)
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver) # Obsolete : tf.distribute.experimental.TPUStrategy()

with strategy.scope(): 
    model = create_model(max_length=300) 

model.load_weights(filepath=checkpoint_path + 'best_bert_weights.h5')

# Classifier as a function

In [ ]:
def predict_sentiment_element(sentence, tokenizer, model):
    """
    가사 내 문장 별 확률 값 추출
    """    
    SEQ_LEN = 300 

    # Tokenizing / Tokens to sequence numbers / Padding
    encoded_dict = tokenizer.encode_plus(text=re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence), # 특수문자 제거
                                         padding='max_length', 
                                         truncation = True,
                                         max_length=SEQ_LEN) 
    
    token_ids = np.array(encoded_dict['input_ids']).reshape(1, -1) 
    token_masks = np.array(encoded_dict['attention_mask']).reshape(1, -1)
    token_segments = np.array(encoded_dict['token_type_ids']).reshape(1, -1)
    
    new_inputs = (token_ids, token_masks, token_segments)

    # Prediction
    pred = model.predict(new_inputs) 
    all_pred_proba = np.round(pred * 100, 2)
    
    #top_pred_proba = np.round(np.max(pred) * 100, 2) 
    #pred_class = ['분노혐오','놀람공포','슬픔','행복'][np.argmax(pred, axis=1)[0]] # 수정하면 4개 클래스 다 나올듯! 
    #print("{}% 확률로 {} 텍스트입니다.".format(top_pred_proba, pred_class))

    return all_pred_proba 

In [ ]:
# def total_sentiment(lyrics):


#### TEST temporary

preprocess_Lyric_ver3에 적용 <br> 

list가 str 형태로 row에 저장되었기 때문에 ast library로 다시 list로 바꿔주어야한다.<br>
(str list to list)

In [ ]:
temp_list = df_allsongs.preprocess_Lyric_ver3.iloc[0]
print(type(temp_list))
temp_list

In [ ]:
import ast  

temp_list_2 = ast.literal_eval(temp_list) 
print(type(temp_list_2)) 
print(temp_list_2)

## str list to list
## sentence = ast.literal_eval(sentence)

In [ ]:
print(len(temp_list_2))
print(temp_list_2[1])
all_pred_proba = predict_sentiment_element(temp_list_2[1], tokenizer, model)
print()
print(all_pred_proba)

In [ ]:
anger = all_pred_proba[0][0]
scary = all_pred_proba[0][1]
sad = all_pred_proba[0][2]
happy = all_pred_proba[0][3]

total = np.ndarray.round( (np.array([(anger, scary, sad, happy)]) / (anger + scary + sad + happy)), 2)

top_pred_prob = total[0][np.argmax(total, axis=1)[0]]

In [ ]:
print(total)
print(top_pred_prob)  

In [ ]:
temp_list_2 = df_allsongs.preprocess_Lyric_ver3.iloc[12]
temp_list_2 = ast.literal_eval(temp_list_2) 
print(len(temp_list_2))
print(temp_list_2[-1])
print(temp_list_2)

In [ ]:
# 마지막 원소 '' 빈칸임. <br/> 태그 기준으로 split해서 나옴, 빈칸인 element 제거
# 가사내 문장 단위로 predict_sentiment_element 함수 적용 
# 앞으로 temp_list_2 자리에 row 넣으면 됨

    # print(len(temp_list_2)) 
    # for i in range(len(temp_list_2)):
        
    #     # 빈칸 제거
    #     if temp_list_2[i] == '':
    #         del temp_list_2[i]  

    #     elif temp_list_2[i] != '':
    #         all_pred_proba = predict_sentiment_element(temp_list_2[i], tokenizer, model)
    #         anger = all_pred_proba[0][0]
    #         scary = all_pred_proba[0][1]
    #         sad = all_pred_proba[0][2]
    #         happy = all_pred_proba[0][3]
    #         #print(i, temp_list_2[i], all_pred_proba) 

    #     anger += anger 
    #     scary += scary
    #     sad  += sad
    #     happy += happy

    # total = np.ndarray.round( (np.array([(anger, scary, sad, happy)]) / (anger + scary + sad + happy)), 2)
    # top_pred_prob = total[0][np.argmax(total, axis=1)[0]]
    # top_pred_class = ['분노혐오','놀람공포','슬픔','행복'][np.argmax(total, axis=1)[0]]

    # print(total)
    # print(top_pred_class) 
    # print(top_pred_prob)
    # print(len(temp_list_2))

In [ ]:
# 마지막 원소 '' 빈칸임. <br/> 태그 기준으로 split해서 나옴, 빈칸인 element 제거
# 가사내 문장 단위로 predict_sentiment_element 함수 적용 
# 앞으로 temp_list_2 자리에 row 넣으면 됨

def predict_sentiment_lyrics(lyrics):
    print(len(lyrics)) 
    for i in range(len(lyrics)):
        
        # 빈칸 제거
        if lyrics[i] == '':
            del lyrics[i]  

        elif lyrics[i] != '':
            all_pred_proba = predict_sentiment_element(lyrics[i], tokenizer, model)
            anger = all_pred_proba[0][0]
            scary = all_pred_proba[0][1]
            sad = all_pred_proba[0][2]
            happy = all_pred_proba[0][3]
            #print(i, lyrics[i], all_pred_proba) 

        anger += anger 
        scary += scary
        sad  += sad
        happy += happy

    total_array = np.ndarray.round( (np.array([(anger, scary, sad, happy)]) / (anger + scary + sad + happy)), 2)
    top_pred_prob = total_array[0][np.argmax(total_array, axis=1)[0]]
    top_pred_class = ['분노혐오','놀람공포','슬픔','행복'][np.argmax(total_array, axis=1)[0]]

    return total_array, top_pred_class, top_pred_prob

# Apply model

In [ ]:
# Emotion = []
# Probability = []
# 분노혐오 = []
# 놀람공포 = []
# 슬픔 = []
# 행복 = []


# for row in tqdm(df_allsongs.iterrows()):

#         pred_class, top_pred_proba, all_pred_proba = predict_sentiment_element(row[1][-1], tokenizer, model)
#         #break
#         Emotion.append(pred_class)
#         Probability.append(top_pred_proba)
#         분노혐오.append(all_pred_proba[0][0])
#         놀람공포.append(all_pred_proba[0][1])
#         슬픔.append(all_pred_proba[0][2])
#         행복.append(all_pred_proba[0][3])

In [ ]:
# df_allsongs['Emotion'] = Emotion
# df_allsongs['Proba'] = Probability
# df_allsongs['분노혐오'] = 분노혐오
# df_allsongs['놀람공포'] = 놀람공포
# df_allsongs['슬픔'] = 슬픔
# df_allsongs['행복'] = 행복

In [ ]:
# df_allsongs['Emotion'].value_counts()

In [ ]:
# df_allsongs

In [ ]:
# df_allsongs.to_csv('songtagged_1.csv')